In [1]:
# First we will import the necessary Library 

import os
import pandas as pd
import numpy as np
import math
from datetime import datetime
from dateutil.relativedelta import relativedelta
import yfinance as yf

# For Evalution we will use these library

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

# For model building we will use these library

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM


# For PLotting we will use these library

import matplotlib.pyplot as plt
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots



In [2]:
today = datetime.today().strftime('%Y-%m-%d')
today


'2022-12-29'

In [3]:
stockStartDate = ('2015-01-01')
stockStartDate

'2015-01-01'

In [4]:
maindf = yf.download('ETH-USD', start= stockStartDate, end= today)
maindf.to_csv('ethereum.csv')

[*********************100%***********************]  1 of 1 completed


In [5]:
maindf = pd.read_csv('ethereum.csv')

In [6]:
print('Total number of days present in the dataset: ',maindf.shape[0])
print('Total number of fields present in the dataset: ',maindf.shape[1])
maindf.shape

Total number of days present in the dataset:  1876
Total number of fields present in the dataset:  7


(1876, 7)

In [7]:
# Printing the start date and End date of the dataset

sd=maindf.iloc[0][0]
ed=maindf.iloc[-1][0]


print('Starting Date',sd)
print('Ending Date',ed)


Starting Date 2017-11-09
Ending Date 2022-12-28


# Analysis of Year 2021

In [8]:
maindf['Date'] = pd.to_datetime(maindf['Date'], format='%Y-%m-%d')

y_2021 = maindf.loc[(maindf['Date'] >= '2021-01-01')
                     & (maindf['Date'] < '2021-12-31')]

y_2021.drop(y_2021[['Adj Close','Volume']],axis=1)

,Date,Open,High,Low,Close
1149,2021-01-01,737.708374,749.201843,719.792236,730.367554
1150,2021-01-02,730.402649,786.798462,718.109497,774.534973
1151,2021-01-03,774.511841,1006.565002,771.561646,975.507690
1152,2021-01-04,977.058838,1153.189209,912.305359,1040.233032
1153,2021-01-05,1041.498779,1129.371460,986.811279,1100.006104
...,...,...,...,...,...
1508,2021-12-26,4094.151611,4105.023926,4013.026367,4067.328125
1509,2021-12-27,4064.746338,4126.001465,4033.492432,4037.547607
1510,2021-12-28,4037.538086,4037.538086,3769.280029,3800.893066
1511,2021-12-29,3797.436279,3827.981934,3612.795898,3628.531738


In [9]:
monthvise= y_2021.groupby(y_2021['Date'].dt.strftime('%B'))[['Open','Close']].mean()
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
monthvise = monthvise.reindex(new_order, axis=0)
monthvise

,Open,Close
Date,,
January,1184.997553,1202.502802
February,1696.234375,1699.808921
March,1720.526655,1736.862222
April,2269.130294,2297.672754
May,3142.513688,3140.730760
June,2349.590800,2335.792391
July,2122.466194,2131.080362
August,3069.708630,3098.384939
September,3349.545020,3337.052686


In [10]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['Open'],
    name='Open Price',
    marker_color='crimson'
))
fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['Close'],
    name='Close Price',
    marker_color='lightsalmon'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Monthwise comparision between open and close price')
fig.show()

In [11]:
y_2021.groupby(y_2021['Date'].dt.strftime('%B'))['Low'].min()
monthvise_high = y_2021.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
monthvise_high = monthvise_high.reindex(new_order, axis=0)

monthvise_low = y_2021.groupby(y_2021['Date'].dt.strftime('%B'))['Low'].min()
monthvise_low = monthvise_low.reindex(new_order, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='High Price',
    marker_color='rgb(0, 153, 204)'
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Low Price',
    marker_color='rgb(255, 128, 0)'
))

fig.update_layout(barmode='group', 
                  title=' Monthwise High and Low price')
fig.show()

In [12]:
# Lets First Take all the Close Price 
closedf = maindf[['Date','Close']]
print("Shape of close dataframe:", closedf.shape)

Shape of close dataframe: (1876, 2)


In [13]:
fig = px.line(closedf, x=closedf.Date, y=closedf.Close,labels={'date':'Date','close':'Close Stock'})
fig.update_traces(marker_line_width=2, opacity=0.8, marker_line_color='orange')
fig.update_layout(title_text='Whole period of timeframe of crypto close price', plot_bgcolor='white', 
                  font_size=15, font_color='black')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

### Now we will Take data of just 1 Year

In [14]:
closedf = closedf[closedf['Date'] > '2021-12-29']
close_stock = closedf.copy()
print("Total data for prediction: ",closedf.shape[0])

Total data for prediction:  364


In [15]:
closedf

,Date,Close
1512,2021-12-30,3713.852051
1513,2021-12-31,3682.632812
1514,2022-01-01,3769.697021
1515,2022-01-02,3829.564941
1516,2022-01-03,3761.380371
...,...,...
1871,2022-12-24,1221.148560
1872,2022-12-25,1218.962036
1873,2022-12-26,1226.974365
1874,2022-12-27,1212.791626


In [16]:
fig = px.line(closedf, x=closedf.Date, y=closedf.Close,labels={'date':'Date','close':'Close Stock'})
fig.update_traces(marker_line_width=2, opacity=0.8, marker_line_color='orange')
fig.update_layout(title_text='Considered period to predict Bitcoin close price', 
                  plot_bgcolor='white', font_size=15, font_color='black')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

* ### Normalizing Data
- Normalization is a technique often applied as part of data preparation for machine learning. The goal of normalization is to change the values of numeric columns in the dataset to use a common scale, without distorting differences in the ranges of values or losing information.
- MinMaxScaler. For each value in a feature, MinMaxScaler subtracts the minimum value in the feature and then divides by the range. The range is the difference between the original maximum and original minimum. MinMaxScaler preserves the shape of the original distribution.

In [17]:
# deleting date column and normalizing using MinMax Scaler

del closedf['Date']
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
print(closedf.shape)

(364, 1)


* ### Slicing data into Training set and Testing set

In [18]:
training_size=int(len(closedf)*0.80)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)

train_data:  (291, 1)
test_data:  (73, 1)


* ### Now we Transform the Close price based on Time-series-analysis forecasting requirement , Here we will take 15   

In [19]:
# convert an array of values into a dataset matrix

def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [20]:
time_step = 15
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)

X_train:  (275, 15)
y_train:  (275,)
X_test:  (57, 15)
y_test (57,)


In [21]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (275, 15, 1)
X_test:  (57, 15, 1)


* # Actuall Model Building

In [22]:
model=Sequential()

model.add(LSTM(10,input_shape=(None,1),activation="relu"))

model.add(Dense(1))

model.compile(loss="mean_squared_error",optimizer="adam")

In [23]:
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=200,batch_size=32,verbose=1)

Epoch 1/200
9/9 [==============================] - 3s 53ms/step - loss: 0.3417 - val_loss: 0.0104
Epoch 2/200
9/9 [==============================] - 0s 12ms/step - loss: 0.2549 - val_loss: 0.0049
Epoch 3/200
9/9 [==============================] - 0s 11ms/step - loss: 0.1866 - val_loss: 0.0023
Epoch 4/200
9/9 [==============================] - 0s 12ms/step - loss: 0.1309 - val_loss: 0.0015
Epoch 5/200
9/9 [==============================] - 0s 12ms/step - loss: 0.0862 - val_loss: 0.0023
Epoch 6/200
9/9 [==============================] - 0s 15ms/step - loss: 0.0515 - val_loss: 0.0045
Epoch 7/200
9/9 [==============================] - 0s 15ms/step - loss: 0.0253 - val_loss: 0.0079
Epoch 8/200
9/9 [==============================] - 0s 15ms/step - loss: 0.0115 - val_loss: 0.0116
Epoch 9/200
9/9 [==============================] - 0s 11ms/step - loss: 0.0083 - val_loss: 0.0132
Epoch 10/200
9/9 [==============================] - 0s 10ms/step - loss: 0.0087 - val_loss: 0.0122
Epoch 11/200
9/9 [=

In [24]:
### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

2/2 [==============================] - 0s 0s/step


((275, 1), (57, 1))

- # Model Evaluation

In [25]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

- # Comparision of original Bitcoin close price and predicted close price

In [26]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['Date'],
                       'original_close': close_stock['Close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (364, 1)
Test predicted data:  (364, 1)


- # Predicting next 30 days

In [27]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 30
while(i<pred_days):
    
    if(len(temp_input)>time_step):
        
        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        
        yhat = model.predict(x_input, verbose=0)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
       
        lst_output.extend(yhat.tolist())
        i=i+1
        
    else:
        
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())
        
        lst_output.extend(yhat.tolist())
        i=i+1
               
print("Output of predicted next days: ", len(lst_output))

Output of predicted next days:  30


- # Plotting last 15 days of dataset and next predicted 30 days

In [28]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39
 40 41 42 43 44 45]


In [29]:
temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

names = cycle(['Last 15 days close price','Predicted next 30 days close price'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Price','index': 'Timestamp'})
fig.update_layout(title_text='Compare last 15 days vs next 30 days',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')

fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()



- # Plotting entire Closing Stock Price with next 30 days period of prediction

In [30]:
next_predicted_days_value

[1309.3287353515625,
 1266.3538818359375,
 1168.2593994140625,
 1188.1495361328125,
 1184.7152099609375,
 1167.60986328125,
 1217.70361328125,
 1213.5999755859375,
 1218.18212890625,
 1220.159423828125,
 1221.1485595703125,
 1218.9620361328125,
 1226.974365234375,
 1212.7916259765625,
 1189.986083984375,
 nan,
 1215.2513504317494,
 1216.2712411605903,
 1216.956887313419,
 1218.1778234396147,
 1219.2685396200513,
 1220.360586947102,
 1221.5245795792493,
 1222.345664617737,
 1223.14737407773,
 1223.8856110864722,
 1224.575926817106,
 1225.236450218757,
 1225.866716446576,
 1226.412465429054,
 1227.0389283808327,
 1227.7968579077392,
 1228.3851613232464,
 1228.945890987461,
 1229.4847306851325,
 1229.9993350627024,
 1230.4912465598982,
 1230.9608877629466,
 1231.4085122235838,
 1231.8367611057265,
 1232.2466697456302,
 1232.639125574371,
 1233.0151216695817,
 1233.3752496519796,
 1233.7204392112635,
 1234.0517890716228]